In [11]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import torch
torch.set_printoptions(precision=10)

from pytorch_lightning import Trainer
from models.ranked_transformer import HsqcRankedTransformer

from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.oneD_dataset import OneDDataModule
from datasets.hsqc_folder_dataset import FolderDataModule

from datasets.optional_2d_folder_dataset import OptionalInputDataModule
import yaml
from pytorch_lightning.loggers import TensorBoardLogger
torch.set_float32_matmul_precision('medium')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")


In [2]:
from datasets.dataset_utils import  fp_loader_configer
fp_loader_configer.select_version("DB_Specific")  #("MFP_Specific_Radius")
fp_loader = fp_loader_configer.fp_loader


choosing DB_Specific_FP_loader


Here we compare models' performance over 7 kinds of inputs


In [ ]:
# load model 

# checkpoint_path = Path("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/stable_argsort/flexible_models_best_FP/r0_r2_FP_trial_1/checkpoints/epoch=32-all_inputs.ckpt")
checkpoint_path = Path("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/rank_on_entire_set/grid_search/select_dim_6144_all_info_trial_3_radius_4/checkpoints/epoch=84-step=22100.ckpt")

model_path = checkpoint_path.parents[1]


hyperpaerameters_path = model_path / "hparams.yaml"

with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
    
FP_building_type = hparams['FP_building_type'].split("_")[-1]
# only_2d = not hparams['use_oneD_NMR_no_solvent']

radius = int(hparams['FP_choice'].split("_")[-1])
fp_loader.setup(out_dim=hparams['out_dim'], max_radius=radius)


del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
# hparams['use_peak_values'] = False




finish entropy list
DB_Specific_FP_loader is setup


In [15]:
args = hparams
datamodule = FolderDataModule(f"/workspace/SMILES_dataset", FP_choice=args["FP_choice"], input_src=["HSQC", "oneD_NMR"], batch_size=args["bs"], parser_args=args) 
# data_module = data_mux(parser, args["modelname"], args["datasrc"], args["FP_choice"], args["bs"], args["ds"], args)
trainer = Trainer( accelerator="auto", accumulate_grad_batches=args["accumulate_grad_batches_num"])
model = HsqcRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)

model.change_ranker_for_testing()



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args


In [16]:
test_result = trainer.test(model, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


loaded full info indices





Testing: |          | 0/? [00:00<?, ?it/s]

{'on_epoch': True, 'sync_dist': False} 




┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                      Test metric                       ┃                      DataLoader 0                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                   test/mean_accuracy                   │                   0.9991598725318909                   │
│                 test/mean_active_bits                  │                   42.380496978759766                   │
│                   test/mean_ce_loss                    │                  0.005044855177402496                  │
│                     test/mean_cos                      │                   0.9276201725006104                   │
│                      test/mean_f1                      │                   0.9367249608039856                   │
│                   test/mean_jaccard                    │                   0.8746678829193115                   │
│                  test/mean_mean_rank                   │                   0.4351213872432709                   │
│                   test/mean_neg_loss                   │                 0.0013249340699985623                  │
│                   test/mean_pos_loss                   │                 0.0037199214566498995                  │
│                 test/mean_pos_neg_loss                 │                  0.005044855643063784                  │
│                  test/mean_precision                   │                   0.9529085159301758                   │
│                    test/mean_rank_1                    │                   0.9423392415046692                   │
│                   test/mean_rank_10                    │                   0.992864191532135                    │
│                    test/mean_rank_5                    │                   0.9896653294563293                   │
│                    test/mean_recall                    │                   0.9212382435798645                   │
│        test/rank_1_of_NP_class/Alkylresorcinols        │                          0.75                          │
│     test/rank_1_of_NP_class/Amino acid glycosides      │                          1.0                           │
│        test/rank_1_of_NP_class/Aminosugars and         │                          1.0                           │
│                    aminoglycosides                     │                                                        │
│   test/rank_1_of_NP_class/Anthranilic acid alkaloids   │                        0.90625                         │
│         test/rank_1_of_NP_class/Apocarotenoids         │                          1.0                           │
│      test/rank_1_of_NP_class/Aromatic polyketides      │                   0.9803921580314636                   │
│       test/rank_1_of_NP_class/Carotenoids (C40)        │                          1.0                           │
│       test/rank_1_of_NP_class/Carotenoids (C50)        │                          1.0                           │
│           test/rank_1_of_NP_class/Chromanes            │                          1.0                           │
│           test/rank_1_of_NP_class/Coumarins            │                   0.9444444179534912                   │
│       test/rank_1_of_NP_class/Cyclic polyketides       │                   0.9056603908538818                   │
│        test/rank_1_of_NP_class/Diarylheptanoids        │                          1.0                           │
│    test/rank_1_of_NP_class/Diazotetronic acids and     │                   0.800000011920929                    │
│                      derivatives                       │                                                        │
│     test/rank_1_of_NP_class/Diphenyl ethers (DPEs)     │                          1.0                           │
│          test/rank_1_of_NP_class/Diterpenoids         

In [25]:
# trainer = Trainer()
trainer = Trainer( accelerator="auto", accumulate_grad_batches=args["accumulate_grad_batches_num"])
trainer.test(model, datamodule, ckpt_path=checkpoint_path) 


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /root/gurusmart/MorganFP_prediction/reproduce_previous_works/stop_on_cosine/all_data_possible/only_c_trial_1/checkpoints/epoch=97-step=79184.ckpt


/workspace/OneD_Only_Dataset/test


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /root/gurusmart/MorganFP_prediction/reproduce_previous_works/stop_on_cosine/all_data_possible/only_c_trial_1/checkpoints/epoch=97-step=79184.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

{'on_epoch': True, 'sync_dist': False} 




┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                      Test metric                       ┃                      DataLoader 0                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                   test/mean_accuracy                   │                   0.9965403079986572                   │
│                 test/mean_active_bits                  │                   161.2662353515625                    │
│                   test/mean_ce_loss                    │                  0.025977211073040962                  │
│                     test/mean_cos                      │                   0.9245877861976624                   │
│                      test/mean_f1                      │                   0.9331362843513489                   │
│                   test/mean_jaccard                    │                   0.8704467415809631                   │
│                  test/mean_mean_rank                   │                   530.9849853515625                    │
│                   test/mean_neg_loss                   │                 0.0048934584483504295                  │
│                   test/mean_pos_loss                   │                  0.021083751693367958                  │
│                 test/mean_pos_neg_loss                 │                  0.025977209210395813                  │
│                  test/mean_precision                   │                   0.9548385739326477                   │
│                    test/mean_rank_1                    │                   0.7517224550247192                   │
│                   test/mean_rank_10                    │                   0.9580872654914856                   │
│                    test/mean_rank_5                    │                   0.9297080039978027                   │
│                    test/mean_recall                    │                     0.91259765625                      │
│        test/rank_1_of_NP_class/Alkylresorcinols        │                          0.75                          │
│     test/rank_1_of_NP_class/Amino acid glycosides      │                   0.5714285969734192                   │
│        test/rank_1_of_NP_class/Aminosugars and         │                   0.6666666865348816                   │
│                    aminoglycosides                     │                                                        │
│   test/rank_1_of_NP_class/Anthranilic acid alkaloids   │                         0.5625                         │
│         test/rank_1_of_NP_class/Apocarotenoids         │                         0.9375                         │
│      test/rank_1_of_NP_class/Aromatic polyketides      │                   0.7058823704719543                   │
│       test/rank_1_of_NP_class/Carotenoids (C40)        │                   0.9090909361839294                   │
│       test/rank_1_of_NP_class/Carotenoids (C50)        │                          1.0                           │
│           test/rank_1_of_NP_class/Chromanes            │                   0.8064516186714172                   │
│           test/rank_1_of_NP_class/Coumarins            │                   0.800000011920929                    │
│       test/rank_1_of_NP_class/Cyclic polyketides       │                   0.698113203048706                    │
│        test/rank_1_of_NP_class/Diarylheptanoids        │                   0.5909090638160706                   │
│    test/rank_1_of_NP_class/Diazotetronic acids and     │                   0.4000000059604645                   │
│                      derivatives                       │                                                        │
│     test/rank_1_of_NP_class/Diphenyl ethers (DPEs)     │                   0.3333333432674408                   │
│          test/rank_1_of_NP_class/Diterpenoids         

[{'test/mean_ce_loss': 0.025977211073040962,
  'test/mean_pos_loss': 0.021083751693367958,
  'test/mean_neg_loss': 0.0048934584483504295,
  'test/mean_pos_neg_loss': 0.025977209210395813,
  'test/mean_cos': 0.9245877861976624,
  'test/mean_jaccard': 0.8704467415809631,
  'test/mean_active_bits': 161.2662353515625,
  'test/mean_f1': 0.9331362843513489,
  'test/mean_precision': 0.9548385739326477,
  'test/mean_recall': 0.91259765625,
  'test/mean_accuracy': 0.9965403079986572,
  'test/mean_mean_rank': 530.9849853515625,
  'test/mean_rank_1': 0.7517224550247192,
  'test/mean_rank_5': 0.9297080039978027,
  'test/mean_rank_10': 0.9580872654914856,
  'test/rank_1_of_NP_class/Saccharides': 0.699999988079071,
  'test/rank_1_of_NP_class/Diterpenoids': 0.8449781537055969,
  'test/rank_1_of_NP_class/Monoterpenoids': 0.8230088353157043,
  'test/rank_1_of_NP_class/Sesquiterpenoids': 0.8509485125541687,
  'test/rank_1_of_NP_class/unknown': 0.6226415038108826,
  'test/rank_1_of_NP_class/Coumarins': 0

In [16]:
trainer = Trainer()
metric_to_focus = [
    'rank_1',
    'rank_5',
    "mean_rank",
    'cos',
    'f1'    
]
def show_model_performance(model, out_file ="temp_model_show.txt"):
    # loaders = [loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR]
    # names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
    # loaders = [ loader_only_1d]
    # names = [ "only_1d"]
    loaders = [loader_only_C_NMR]
    names = ["loader_only_C_NMR"]
    # names = ["only_hsqc"]
    with open(out_file, "w") as file:
        for loader, name in zip(loaders, names):
            result =  trainer.test(model, dataloaders=loader, ckpt_path=checkpoint_path) 
            # result = trainer.test(model, datamodule, ckpt_path = checkpoint_path)
            file.write(f"\n{name:10}: ")
            for m in metric_to_focus:
                file.write(f"{m}: {result[0]['test/mean_'+m]:.4f}, ")
    return result

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [17]:
result = show_model_performance(model)


Restoring states from the checkpoint path at /root/gurusmart/MorganFP_prediction/reproduce_previous_works/stop_on_cosine/all_data_possible/only_c_trial_1/checkpoints/epoch=97-step=79184.ckpt
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:277: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': 'val/mean_cos', 'mode': 'max', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None}", "EarlyStopping{'monitor': 'val/mean_cos', 'mode': 'max'}"].
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /root/gurusmart/MorganFP_prediction/reproduce_previous_works/stop_on_cosine/all_data_possible/only_c_trial_1/checkpoints/epoch=97-step=79184.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

{'on_epoch': True, 'sync_dist': False} 




┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                      Test metric                       ┃                      DataLoader 0                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                   test/mean_accuracy                   │                   0.9965403079986572                   │
│                 test/mean_active_bits                  │                   161.2662353515625                    │
│                   test/mean_ce_loss                    │                  0.025977211073040962                  │
│                     test/mean_cos                      │                   0.9245877861976624                   │
│                      test/mean_f1                      │                   0.9331362843513489                   │
│                   test/mean_jaccard                    │                   0.8704467415809631                   │
│                  test/mean_mean_rank                   │                   530.9849853515625                    │
│                   test/mean_neg_loss                   │                 0.0048934584483504295                  │
│                   test/mean_pos_loss                   │                  0.021083751693367958                  │
│                 test/mean_pos_neg_loss                 │                  0.025977209210395813                  │
│                  test/mean_precision                   │                   0.9548385739326477                   │
│                    test/mean_rank_1                    │                   0.7517224550247192                   │
│                   test/mean_rank_10                    │                   0.9580872654914856                   │
│                    test/mean_rank_5                    │                   0.9297080039978027                   │
│                    test/mean_recall                    │                     0.91259765625                      │
│        test/rank_1_of_NP_class/Alkylresorcinols        │                          0.75                          │
│     test/rank_1_of_NP_class/Amino acid glycosides      │                   0.5714285969734192                   │
│        test/rank_1_of_NP_class/Aminosugars and         │                   0.6666666865348816                   │
│                    aminoglycosides                     │                                                        │
│   test/rank_1_of_NP_class/Anthranilic acid alkaloids   │                         0.5625                         │
│         test/rank_1_of_NP_class/Apocarotenoids         │                         0.9375                         │
│      test/rank_1_of_NP_class/Aromatic polyketides      │                   0.7058823704719543                   │
│       test/rank_1_of_NP_class/Carotenoids (C40)        │                   0.9090909361839294                   │
│       test/rank_1_of_NP_class/Carotenoids (C50)        │                          1.0                           │
│           test/rank_1_of_NP_class/Chromanes            │                   0.8064516186714172                   │
│           test/rank_1_of_NP_class/Coumarins            │                   0.800000011920929                    │
│       test/rank_1_of_NP_class/Cyclic polyketides       │                   0.698113203048706                    │
│        test/rank_1_of_NP_class/Diarylheptanoids        │                   0.5909090638160706                   │
│    test/rank_1_of_NP_class/Diazotetronic acids and     │                   0.4000000059604645                   │
│                      derivatives                       │                                                        │
│     test/rank_1_of_NP_class/Diphenyl ethers (DPEs)     │                   0.3333333432674408                   │
│          test/rank_1_of_NP_class/Diterpenoids         

In [ ]:
result

[{'test/mean_ce_loss': 0.17407427728176117,
  'test/mean_pos_loss': 0.1728496253490448,
  'test/mean_neg_loss': 0.0012246511178091168,
  'test/mean_pos_neg_loss': 0.17407427728176117,
  'test/mean_cos': 0.5992004871368408,
  'test/mean_jaccard': 0.38752874732017517,
  'test/mean_active_bits': 65.92560577392578,
  'test/mean_f1': 0.5462456345558167,
  'test/mean_precision': 0.9684531688690186,
  'test/mean_recall': 0.3839390277862549,
  'test/mean_accuracy': 0.9826473593711853,
  'test/mean_mean_rank': 5966.78076171875,
  'test/mean_rank_1': 0.11154855787754059,
  'test/mean_rank_5': 0.23072506487369537,
  'test/mean_rank_10': 0.2965058982372284,
  'test/rank_1_of_NP_class/Saccharides': 0.10000000149011612,
  'test/rank_1_of_NP_class/Diterpenoids': 0.07205240428447723,
  'test/rank_1_of_NP_class/Monoterpenoids': 0.1150442510843277,
  'test/rank_1_of_NP_class/Sesquiterpenoids': 0.1490514874458313,
  'test/rank_1_of_NP_class/unknown': 0.09164420515298843,
  'test/rank_1_of_NP_class/Coumar

In [ ]:
# spectra-flexible model
# usually not gonna use it because it tests right away after training 

# load model 

# model_path = Path("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1")
# hyperpaerameters_path = model_path / "hparams.yaml"

# checkpoint_path = model_path / "checkpoints/epoch=41-step=35994.ckpt"
# with open(hyperpaerameters_path, 'r') as file:
#     hparams = yaml.safe_load(file)
# del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
# hparams['use_peak_values'] = False

# model = OptionalInputRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
# model.change_ranker_for_testing()

# flexible_model_result = trainer.test(model, datamodule)


FileNotFoundError: [Errno 2] No such file or directory: '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1/hparams.yaml'

In [ ]:

# names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
# out_file = "temp_model_show.txt"
# with open(out_file, "w") as file:
#     for name in  names:
#         file.write(f"\n{name:10}: ")
#         for m in metric_to_focus:
#             file.write(f"{m}: {flexible_model_result[0]['test_mean_'+m+'_'+name]:.4f}, ")


In [ ]:
# flexible_model_result[0] .keys()    


dict_keys(['test_mean_ce_loss_all_inputs', 'test_mean_pos_loss_all_inputs', 'test_mean_neg_loss_all_inputs', 'test_mean_pos_neg_loss_all_inputs', 'test_mean_cos_all_inputs', 'test_mean_active_bits_all_inputs', 'test_mean_f1_all_inputs', 'test_mean_precision_all_inputs', 'test_mean_recall_all_inputs', 'test_mean_accuracy_all_inputs', 'test_mean_mean_rank_all_inputs', 'test_mean_rank_1_all_inputs', 'test_mean_rank_5_all_inputs', 'test_mean_rank_10_all_inputs', 'test_mean_ce_loss_HSQC_H_NMR', 'test_mean_pos_loss_HSQC_H_NMR', 'test_mean_neg_loss_HSQC_H_NMR', 'test_mean_pos_neg_loss_HSQC_H_NMR', 'test_mean_cos_HSQC_H_NMR', 'test_mean_active_bits_HSQC_H_NMR', 'test_mean_f1_HSQC_H_NMR', 'test_mean_precision_HSQC_H_NMR', 'test_mean_recall_HSQC_H_NMR', 'test_mean_accuracy_HSQC_H_NMR', 'test_mean_mean_rank_HSQC_H_NMR', 'test_mean_rank_1_HSQC_H_NMR', 'test_mean_rank_5_HSQC_H_NMR', 'test_mean_rank_10_HSQC_H_NMR', 'test_mean_ce_loss_HSQC_C_NMR', 'test_mean_pos_loss_HSQC_C_NMR', 'test_mean_neg_loss_